In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get('riot_api_key')


In [2]:
import pygsheets 
google_url = os.environ.get('google_url')
service_account = pygsheets.authorize(service_account_file = 'JSONS\\spreadsheet-automator-447604-0758cb3da318.json')
sheet = service_account.open_by_url(google_url)

In [3]:
import pandas as pd 
import requests
import time
def getLadder(rank,division):
    divisions = ['II','III','IV']
    page = pd.DataFrame()
    root = 'https://na1.api.riotgames.com/'
    meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page=1'
    api_url = root + meta + '&api_key=' + api_key
    api_response = requests.get(api_url)
    api_df = pd.DataFrame(api_response.json())
    page = pd.concat([page,api_df])
    page_num = 2 
    while not api_df.empty:
        meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page={page_num}'
        api_url = root + meta + '&api_key=' + api_key
        api_response = requests.get(api_url)
        api_df = pd.DataFrame(api_response.json())
        page = pd.concat([page,api_df])
        page_num+=1
        time.sleep(1)
    return page

In [4]:
def get_puuid(summoner_id):
    root = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/'
    api_url = root + summoner_id + '?api_key=' + api_key
    page = requests.get(api_url)
    return page.json()['puuid']

In [25]:
if os.path.exists('ironLadder.csv'):
    ironLadder = pd.read_csv('ironLadder.csv')
else:
    ironLadder = getLadder('IRON','I').sort_values('leaguePoints',ascending=False).reset_index(drop=True)
    ironLadder.to_csv('ironLadder.csv',index=False)
    print('Not found')

In [18]:
def getMatch(matchID):
    x = 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5206478055?api_key=RGAPI-3418b2cb-2001-4b9a-8409-e3bbd2a04989'
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/'
    middle = '?api_key='
    api_url = root + matchID + middle + api_key
    return requests.get(api_url).json()

In [16]:
def getListMatches(puuid,start,end):
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/'
    middle = '/ids?type=ranked&'
    part = 'start='+str(start)+'&count='+str(end)
    api_url = root +str(puuid)+middle + part + '&api_key=' + api_key
    page = requests.get(api_url).json()
    return page 

In [ ]:
def getKDA():
    x= 0

In [ ]:
def getCS():
    x= 0 

In [29]:
# For each person in the ladder, look at their 0 - 10 games and get their average cs, KDA
average_iron_1 = {'puuid':[],'KDA':[],'CS':[]}
for i in ironLadder.itertuples():
    

9
12
4
11
6
6
6
6
6
6
9
6
4
6
6
17
12
4
8
6
6
6
9
6
6
10
6
12
8
8
8
11
4
12
4
10
6
6
4
6
6
9
10
10
6
10
6
4
6
6
8
2
6
4
6
6
4
6
10
6
6
6
6
4
6
6
6
14
14
6
4
6
6
9
6
4
7
6
7
9
6
16
6
10
4
15
10
6
10
6
11
13
6
6
8
10
8
18
6
12
6
8
10
13
12
13
9
4
6
10
10
26
10
15
6
11
30
16
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
10
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
14
3
3
3
3
3
3
3
3
3
10
3
9
3
3
5
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
15
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
12
8
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
5
3
3
3
10
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
11
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
14
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
7
3
3
3
3
3
3
3
3
8
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
14
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
11
3
3
3
3
3
3
3
3
3
3
3
3
3
12
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
3
3
3
15
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3